## GPT 4O 圖片串接

In [15]:
import os
import base64
import openai

openai.api_key = "YOUR OPENAI API"
MODEL="gpt-4o"

def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

# Path to your image
image_path = "pic/bandon.jpg"

# Getting the base64 string
base64_image = encode_image(image_path)

response = openai.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": "你是一名擅長估計熱量的營養師. 請幫我計算熱量!"},
        {"role": "user", "content": [
            {"type": "text", "text": "這些大該多少卡路里?以及告訴我這些菜具體是什麼菜?"},
            {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}
            }
        ]}
    ],
    temperature=0.0,
)

print(response.choices[0].message.content)


這是一個典型的台式便當，裡面包含以下幾道菜：

1. **滷雞腿**：這是一隻滷製的雞腿，通常會有醬油、糖、五香粉等調料。大約有250-300卡路里。
2. **白飯**：大約一碗白飯，約200-250卡路里。
3. **炒青菜**：這裡看起來像是炒青江菜，約50-70卡路里。
4. **炒粉絲**：這裡的粉絲看起來是用胡蘿蔔和洋蔥炒的，約100-150卡路里。
5. **滷豆腐**：這裡有幾塊滷製的豆腐，約50-70卡路里。

總熱量估計：
- 滷雞腿：250-300卡路里
- 白飯：200-250卡路里
- 炒青菜：50-70卡路里
- 炒粉絲：100-150卡路里
- 滷豆腐：50-70卡路里

總計：約650-840卡路里。

這只是大概的估計，實際熱量可能會因為烹飪方法和食材的不同而有所變化。


## GPT + yolo segment 準備圖片資料庫

In [2]:
import tiktoken

def num_tokens_from_string(string: str, encoding_name: str) -> int:
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

print(num_tokens_from_string("Hello world, let's test tiktoken.", "cl100k_base"))

9


In [3]:
from ultralytics import YOLO
import random
import cv2
import numpy as np
import tiktoken

In [4]:
model = YOLO("yolov8m-seg.pt")

In [5]:
import os
import base64
import openai
import json
import pandas as pd

csv_file = 'detected_objects.csv'
csv_columns = ['id', 'name', 'calorie', 'grams', 'image']

def num_tokens_from_string(string: str, encoding_name: str) -> int:
    encoding = tiktoken.encoding_for_model(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens


def modify_string(s):
    # 检查输入是否为字符串类型
    if not isinstance(s, str):
        try:
            s = s.text  # 尝试将输入转换为文本属性
        except AttributeError:
            raise ValueError("Input is not a string and does not have a 'text' attribute")
    
    # 查找字符串中第一个 '{' 和最后一个 '}' 的位置
    start_index = s.find('{')
    end_index = s.rfind('}')
    
    # 如果找到并且 '{' 在 '}' 之前，截取子字符串
    if start_index != -1 and end_index != -1 and start_index < end_index:
        s = s[start_index:end_index + 1]
    else:
        s = ''  # 否则返回空字符串
    
    # 将结果用方括号框住
    return f'[{s}]'

openai.api_key = "YOUR OPENAI API"
MODEL="gpt-4o"
pic = "pic/dinner.png"


img = cv2.imread(pic)
# if you want all classes
yolo_classes = list(model.names.values())
classes_ids = [yolo_classes.index(clas) for clas in yolo_classes]
ori_img = cv2.imread(pic)
conf = 0.1
 
results = model.predict(img, conf=conf)
colors = [random.choices(range(256), k=3) for _ in classes_ids]
print(results)
for result in results:
    for mask, box in zip(result.masks.xy, result.boxes):
        points = np.int32([mask])
        # cv2.polylines(img, points, True, (255, 0, 0), 1)
        color_number = classes_ids.index(int(box.cls[0]))
        cv2.fillPoly(img, points, colors[color_number])

# 遍历每个预测结果
for result_idx, result in enumerate(results):
    for mask, box in zip(result.masks.xy, result.boxes):
        points = np.int32([mask])
        color_number = classes_ids.index(int(box.cls[0]))

        # 填充多边形
        cv2.fillPoly(img, points, colors[color_number])

        # 获取边界框坐标
        x_min, y_min, x_max, y_max = map(int, box.xyxy[0])
        
        # 裁剪出每个物体
        object_img = ori_img[y_min:y_max, x_min:x_max]
        _, buffer = cv2.imencode('.jpg', object_img)
        object_img_encode = base64.b64encode(buffer).decode('utf-8')

        messages_prompt = [
        {"role": "system", "content": "You are a nutritionist skilled in estimating calories. Please help me calculate the calories!"},
        {"role": "user", "content": [
            {"type": "text", "text": "Please return the result in JSON format with the corresponding JSON keys being name, calorie, Grams"},
            {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{object_img_encode}"}}
        ]}
        ]
        m = "You are a nutritionist skilled in estimating calories. Please help me calculate the calories! Please return the result in JSON format with the corresponding JSON keys being name, calorie, Grams."

        # Convert the messages list to a JSON string
        messages_string = json.dumps(messages_prompt, ensure_ascii=False)

        # Use the correct encoding name
        tok = num_tokens_from_string(m, "gpt-4o")
        print(f"運算價格:${(tok/1000)*0.06}")


        response = openai.chat.completions.create(
        model=MODEL,
        messages=messages_prompt, 
        temperature=0.0,)
        # 保存裁剪后的图像
        modified_string = modify_string(response.choices[0].message.content)
        print(modified_string)
        itinerary_json = json.loads(modified_string)

        df = pd.json_normalize(itinerary_json)
        df['image'] = f"object_{result_idx}_{color_number}.png"
        cv2.imwrite(f"object_{result_idx}_{color_number}.png", object_img)
        df.to_csv('detected_objects.csv', mode='a', header=False, index=False)



0: 416x640 1 bottle, 2 spoons, 3 bowls, 1 broccoli, 1 hot dog, 1 dining table, 184.1ms
Speed: 6.0ms preprocess, 184.1ms inference, 2025.8ms postprocess per image at shape (1, 3, 416, 640)
[ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
masks: ultralytics.engine.results.Masks object
names: {0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork'